In [ ]:
# Find existing Asset and create an attachment
#
# Main function, establishes a connection to RKVST using an App Registration then uses that 
# to find existing Asset and create an attachment.
#
# The attachment is an image located in a folder named "test_files"

In [ ]:
from json import dumps as json_dumps
from os import getenv
from warnings import filterwarnings

from archivist.archivist import Archivist
from archivist.proof_mechanism import ProofMechanism
from archivist.logger import set_logger

filterwarnings("ignore", message="Unverified HTTPS request")

try:
    import importlib.resources as pkg_resources
except ImportError:
    import importlib_resources as pkg_resources
    
import test_files

In [ ]:
# Connection parameters.  Fill in with client ID and secret from an appropriately 
# permissioned Application Registration
RKVST_URL="https://app.rkvst.io"
# APPREG_CLIENT="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
# APPREG_SECRET="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
def upload_attachment(arch, path, name):
    """
    Obtains attachments from "test_files" folder and creates json payload
    for upload
    """
    with pkg_resources.open_binary(test_files, path) as fd:
        blob = arch.attachments.upload(fd)
        attachment = {
            "arc_display_name": name,
            "arc_attachment_identity": blob["identity"],
            "arc_hash_value": blob["hash"]["value"],
            "arc_hash_alg": blob["hash"]["alg"]
        }
        return attachment

In [ ]:
def get_artist(arch, name, artistid):
    """
    Finds existing Artist asset by name and unique id
    """
    attrs = {
        "arc_display_type": "Artists",
        "arc_display_name": name,
        "artistid": artistid
    }
    
    return arch.assets.read_by_signature(attrs=attrs)

In [ ]:
def create_image(arch, asset):
    """
    Creates an event that uploads a primary image for an Artist asset
    """
    attachments = upload_attachment(arch, "pexels-andrea-turner-707697.jpeg", "arc_primary_image")
    attachment_list = [attachments]
    
    props = {
        "operation": "Record",
        "behaviour": "RecordEvidence"
    }
    attrs = {
        "arc_description": "Attaching an image",
        "arc_display_type": "Primary image",
    }
    
    asset_attrs = {
        "arc_attachments": attachment_list
    }
    
    return arch.events.create(asset["identity"], props=props, attrs=attrs, asset_attrs=asset_attrs)

In [ ]:
"""
Main function of asset and event creation.

* Connect to RKVST with client ID and client secret
* Finds existing asset
* Creates additional event for attachment upload
* Prints response of asset and event creation
"""
# Optional call to set the logger level.  The argument can be either 
# "INFO" or "DEBUG".  For more sophisticated logging control see our 
# documentation.
set_logger("INFO")

# Initialize connection to RKVST
print("Connecting to RKVST")
arch = Archivist(RKVST_URL, (APPREG_CLIENT, APPREG_SECRET), max_time=300)

In [ ]:
# Finding existing artist by name and artist id
# Fill in the artistid from Create Asset and Events notebook
print("Finding Asset")
asset = get_artist(arch, "Adele Laurie Blue Adkins","xxxxxxxxxxxx")
print("Asset", json_dumps(asset, indent=4))

In [ ]:
# Create an event for attachment upload
print("Creating Events for existing Asset")
image_event = create_image(arch, asset)
print("Event for Image", json_dumps(image_event, indent=4))